In [17]:
import tkinter as tk
from tkinter import messagebox
import mysql.connector


def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="SansAmu@0603",
        database="project"
    )


def login():
    username = ename.get()
    password = epassword.get()

    conn = connect_to_db()
    cursor = conn.cursor()
    query = "SELECT * FROM users WHERE username = %s AND password = %s"
    cursor.execute(query, (username, password))
    result = cursor.fetchone()

    if result:
        messagebox.showinfo("Login Successful", "Welcome, " + username + "!")
        show_todo_list(username)
    else:
        messagebox.showerror("Login Failed", "Incorrect username or password")

    cursor.close()
    conn.close()


def register():
    username = ename_reg.get()
    email = eemail_reg.get()
    phone = ephone_reg.get()
    password = epassword_reg.get()
    confirm_password = econfirm_password_reg.get()

    if password != confirm_password:
        messagebox.showerror("Error", "Passwords do not match!")
        return

    conn = connect_to_db()
    cursor = conn.cursor()

    query = "INSERT INTO users (username, email, phone, password) VALUES (%s, %s, %s, %s)"
    try:
        cursor.execute(query, (username, email, phone, password))
        conn.commit()
        messagebox.showinfo("Success", "Registration successful!")
        show_todo_list(username)
    except mysql.connector.Error as err:
        messagebox.showerror("Error", str(err))
    finally:
        cursor.close()
        conn.close()


def open_registration():
    login_frame.pack_forget()
    registration_frame.pack()


def back_to_login():
    registration_frame.pack_forget()
    login_frame.pack()


def show_todo_list(username):
    global todo_list_app
    login_frame.pack_forget()
    registration_frame.pack_forget()
    todo_list_app = TodoList(username)
    todo_list_frame.pack()


class TodoList:
    def __init__(self, username):
        self.username = username
        self.tasks = []

        self.task_entry = tk.Entry(todo_list_frame, width=30, font=('Times New Roman', 12))
        self.task_entry.grid(row=0, column=0, padx=10, pady=10)

        add_button = tk.Button(todo_list_frame, text="Add Task", command=self.add_task, font=('Times New Roman', 12),
                               bg="lightsteelblue1", fg="black")
        add_button.grid(row=0, column=1, padx=10, pady=10)

        self.task_listbox = tk.Listbox(todo_list_frame, width=40, height=20, font=('Times New Roman', 12),
                                       bg="#ecf0f1", fg="black", selectbackground="lightsteelblue3")
        self.task_listbox.grid(row=1, column=0, columnspan=2, padx=10, pady=10)

        remove_button = tk.Button(todo_list_frame, text="Remove Task", command=self.remove_task, font=('Times New Roman', 12),
                                  bg="#e74c3c", fg="black")
        remove_button.grid(row=2, column=0, padx=10, pady=10, sticky="ew")

        complete_button = tk.Button(todo_list_frame, text="Complete Task", command=self.complete_task, font=('Times New Roman', 12),
                                    bg="yellow", fg="black")
        complete_button.grid(row=2, column=1, padx=10, pady=10, sticky="ew")

        self.task_listbox.bind('<Double-Button-1>', lambda event: self.complete_task())

        self.load_tasks()

    def connect_to_db(self):
        return mysql.connector.connect(
            host="localhost",
            user="root",
            password="SansAmu@0603",
            database="project"
        )

    def load_tasks(self):
        conn = self.connect_to_db()
        cursor = conn.cursor()
        query = "SELECT task_name FROM tasks WHERE username = %s AND task_status = 'InProgress'"
        cursor.execute(query, (self.username,))
        for (task_name,) in cursor.fetchall():
            self.tasks.append(task_name)
            self.task_listbox.insert(tk.END, task_name)
        cursor.close()
        conn.close()

    def add_task(self):
        task = self.task_entry.get()
        if task:
            conn = self.connect_to_db()
            cursor = conn.cursor()
            query = "INSERT INTO tasks (username, task_name, task_status) VALUES (%s, %s, 'InProgress')"
            cursor.execute(query, (self.username, task))
            conn.commit()
            cursor.close()
            conn.close()

            self.tasks.append(task)
            self.update_task_list()
            self.task_entry.delete(0, tk.END)
        else:
            messagebox.showwarning("Warning", "Please enter a task.")

    def remove_task(self):
        selected_task_index = self.task_listbox.curselection()
        if selected_task_index:
            task_name = self.tasks.pop(selected_task_index[0])

            conn = self.connect_to_db()
            cursor = conn.cursor()
            query = "DELETE FROM tasks WHERE username = %s AND task_name = %s"
            cursor.execute(query, (self.username, task_name))
            conn.commit()
            cursor.close()
            conn.close()

            self.update_task_list()

    def complete_task(self):
        selected_task_index = self.task_listbox.curselection()
        if selected_task_index:
            task_name = self.tasks[selected_task_index[0]]

            conn = self.connect_to_db()
            cursor = conn.cursor()
            query = "UPDATE tasks SET task_status = 'Done' WHERE username = %s AND task_name = %s"
            cursor.execute(query, (self.username, task_name))
            conn.commit()
            cursor.close()
            conn.close()

            self.tasks[selected_task_index[0]] = f"{task_name} (Done)"
            self.update_task_list()

    def update_task_list(self):
        self.task_listbox.delete(0, tk.END)
        for task in self.tasks:
            self.task_listbox.insert(tk.END, task)


root = tk.Tk()
root.title("Login and To-Do List")
root.config(bg="lavenderblush1")

login_frame = tk.Frame(root, bg="lavenderblush1")

name_label = tk.Label(login_frame, text="Username", bg="lavenderblush1", font=('Times New Roman', 12))
name_label.pack(pady=5)
ename = tk.Entry(login_frame, font=('Times New Roman', 12))
ename.pack(pady=5)

password_label = tk.Label(login_frame, text="Password", bg="lavenderblush1", font=('Times New Roman', 12))
password_label.pack(pady=5)
epassword = tk.Entry(login_frame, show="*", font=('Times New Roman', 12))
epassword.pack(pady=5)

button_login = tk.Button(login_frame, text="Login", command=login, bg="lightsteelblue1", fg="black",
                         font=('Times New Roman', 12, 'bold'))
button_login.pack(pady=10)

register_label = tk.Label(login_frame, text="If not registered, click below:", bg="lavenderblush1",
                          font=('Times New Roman', 12))
register_label.pack(pady=5)
button_register = tk.Button(login_frame, text="Register here", command=open_registration, bg="lightsteelblue1", fg="black",
                            font=('Times New Roman', 12, 'bold'))
button_register.pack(pady=5)

registration_frame = tk.Frame(root, bg="lavenderblush1")

name_label_reg = tk.Label(registration_frame, text="Username", bg="lavenderblush1", font=('Times New Roman', 12))
name_label_reg.pack(pady=5)
ename_reg = tk.Entry(registration_frame, font=('Times New Roman', 12))
ename_reg.pack(pady=5)

email_label_reg = tk.Label(registration_frame, text="Email", bg="lavenderblush1", font=('Times New Roman', 12))
email_label_reg.pack(pady=5)
eemail_reg = tk.Entry(registration_frame, font=('Times New Roman', 12))
eemail_reg.pack(pady=5)

phone_label_reg = tk.Label(registration_frame, text="Phone Number", bg="lavenderblush1", font=('Times New Roman', 12))
phone_label_reg.pack(pady=5)
ephone_reg = tk.Entry(registration_frame, font=('Times New Roman', 12))
ephone_reg.pack(pady=5)

password_label_reg = tk.Label(registration_frame, text="Password", bg="lavenderblush1", font=('Times New Roman', 12))
password_label_reg.pack(pady=5)
epassword_reg = tk.Entry(registration_frame, show="*", font=('Times New Roman', 12))
epassword_reg.pack(pady=5)

confirm_password_label_reg = tk.Label(registration_frame, text="Confirm Password", bg="lavenderblush1",
                                      font=('Times New Roman', 12))
confirm_password_label_reg.pack(pady=5)
econfirm_password_reg = tk.Entry(registration_frame, show="*", font=('Times New Roman', 12))
econfirm_password_reg.pack(pady=5)

button_register_reg = tk.Button(registration_frame, text="Register", command=register, bg="lightsteelblue1", fg="black",
                                font=('Times New Roman', 12, 'bold'))
button_register_reg.pack(pady=10)

button_back = tk.Button(registration_frame, text="Back to Login", command=back_to_login, bg="lightsteelblue1", fg="black",
                        font=('Times New Roman', 12, 'bold'))
button_back.pack(pady=10)

todo_list_frame = tk.Frame(root, bg="lavenderblush1")
todo_list_app = None

login_frame.pack()
root.geometry("400x600")

root.mainloop()
